# Wind speed inversion from level-1 product 

In [ ]:
import xsarsea
from xsarsea import windspeed
import xarray as xr
import numpy as np
import holoviews as hv
hv.extension('bokeh')

import os,sys,re, cv2

In [ ]:
# optional debug messages
#import logging
#logging.basicConfig()
#logging.getLogger('xsarsea.windspeed').setLevel(logging.DEBUG) # or .setLevel(logging.INFO)

## Requirements for inversion
xsar is required

In [ ]:
import xsar

Getting metadata

In [ ]:
safe_path = xsarsea.get_test_file("S1A_EW_GRDM_1SDV_20230217T002336_20230217T002412_047268_05AC30_Z005.SAFE")
s1meta = xsar.Sentinel1Meta(safe_path)

land mask:
not applied yet 

getting associated ancillary data (ecmwf)

In [ ]:
s1meta.set_raster('ecmwf_0100_1h','/home/datawork-cersat-public/provider/ecmwf/forecast/hourly/0100deg/netcdf_light/%Y/%j/ECMWF_FORECAST_0100_%Y%m%d%H%M_10U_10V.nc')
import datetime
for ecmwf_name in ['ecmwf_0100_1h' ]:
    ecmwf_infos = s1meta.rasters.loc[ecmwf_name]
    ecmwf_file = ecmwf_infos['get_function'](ecmwf_infos['resource'], date=datetime.datetime.strptime(s1meta.start_date, '%Y-%m-%d %H:%M:%S.%f'))
    ecmwf_file = xsarsea.get_test_file(ecmwf_file.split('/')[-1],iszip=False)
    map_model = { '%s_%s' % (ecmwf_name, uv) : 'model_%s' % uv for uv in ['U10', 'V10'] }

In [ ]:
map_model

In [ ]:
s1meta.rasters.at["ecmwf_0100_1h","resource"] = ecmwf_file

Mapping model & adding ancillary wind 

In [ ]:
### Loading dataset & merging ancillary
xsar_obj_1000m = xsar.Sentinel1Dataset(s1meta, resolution='1000m')

In [ ]:
dataset_1000m = xsar_obj_1000m.datatree['measurement'].to_dataset()
dataset_1000m = dataset_1000m.rename(map_model)

creation of variables of interest for inversion 

here we could add a land/ice mask.

In [ ]:
### Variables of interest 
#xsar_obj_1000m.dataset['land_mask'].values = cv2.dilate(xsar_obj_1000m.dataset['land_mask'].values.astype('uint8'),np.ones((3,3),np.uint8),iterations = 3)
#xsar_obj_1000m.dataset['sigma0_ocean'] = xr.where(xsar_obj_1000m.dataset['land_mask'], np.nan, xsar_obj_1000m.dataset['sigma0'].compute()).transpose(*xsar_obj_1000m.dataset['sigma0'].dims)
#xsar_obj_1000m.dataset['sigma0_ocean'] = xr.where(xsar_obj_1000m.dataset['sigma0_ocean'] <= 0, 1e-15, xsar_obj_1000m.dataset['sigma0_ocean'])

In [ ]:
dataset_1000m['sigma0_ocean'] = xr.where(dataset_1000m['sigma0'] <= 0, 1e-15, xsar_obj_1000m.dataset['sigma0'])
dataset_1000m['ancillary_wind'] = (dataset_1000m.model_U10 + 1j * dataset_1000m.model_V10) * np.exp(1j * np.deg2rad(dataset_1000m.ground_heading))
dataset_1000m['ancillary_wind'] = xr.where(dataset_1000m['land_mask'], np.nan, dataset_1000m['ancillary_wind'].compute()).transpose(*dataset_1000m['ancillary_wind'].dims)
dataset_1000m.attrs['ancillary_source'] = dataset_1000m['model_U10'].attrs['history'].split('decoded: ')[1].strip()

In [ ]:
hv.Image(dataset_1000m['sigma0_ocean'].sel(pol='VH')).opts(colorbar=True,cmap='binary',width=125, height=100, tools = ['hover'], title = "sigma0 VH")

## Inversion

### inversion parameters

In [ ]:
apply_flattening = True
GMF_VH_NAME = "gmf_s1_v2"

apply flattening or not

In [ ]:
nesz_cr = dataset_1000m.nesz.isel(pol=1) #(no_flattening)
if apply_flattening : 
    dataset_1000m=dataset_1000m.assign(nesz_VH_final=(['line','sample'],windspeed.nesz_flattening(nesz_cr, dataset_1000m.incidence)))
    dataset_1000m['nesz_VH_final'].attrs["comment"] = 'nesz has been flattened using windspeed.nesz_flattening'
else :
    dataset_1000m=dataset_1000m.assign(nesz_VH_final=(['line','sample'],nesz_cr.values))
    dataset_1000m['nesz_VH_final'].attrs["comment"] = 'nesz has not been flattened'

compute dsig_cr (mix between polarisations) using the last version : "gmf_s1_v2"

In [ ]:
dsig_cr = windspeed.get_dsig("gmf_s1_v2", dataset_1000m.incidence,dataset_1000m.sigma0_ocean.sel(pol='VH'),dataset_1000m.nesz_VH_final)

### get windspeed in dfferent polarizations

CO and DUAL

In [ ]:
windspeed_co, windspeed_dual = windspeed.invert_from_model(
        dataset_1000m.incidence,
        dataset_1000m.sigma0_ocean.isel(pol=0),
        dataset_1000m.sigma0_ocean.isel(pol=1),
        #ancillary_wind=-np.conj(xsar_obj_1000m.dataset['ancillary_wind']),
        ancillary_wind=-dataset_1000m['ancillary_wind'],
        dsig_cr = dsig_cr,
        model=('cmod5n',GMF_VH_NAME))

dataset_1000m["windspeed_co"] = np.abs(windspeed_co)
dataset_1000m["windspeed_co"].attrs["comment"] = dataset_1000m["windspeed_co"].attrs["comment"].replace("wind speed and direction","wind speed")
dataset_1000m["windspeed_dual"] = np.abs(windspeed_dual)
dataset_1000m["windspeed_dual"].attrs["comment"] = dataset_1000m["windspeed_dual"].attrs["comment"].replace("wind speed and direction","wind speed")

CR

In [ ]:
windspeed_cr = windspeed.invert_from_model(
    dataset_1000m.incidence.values,
    dataset_1000m.sigma0_ocean.isel(pol=1).values,
    #ancillary_wind=-np.conj(xsar_obj_1000m.dataset['ancillary_wind']),
    dsig_cr = dsig_cr.values,
    model=GMF_VH_NAME)

windspeed_cr = np.abs(windspeed_cr)
dataset_1000m=dataset_1000m.assign(windspeed_cr=(['line','sample'],windspeed_cr))
dataset_1000m.windspeed_cr.attrs['comment'] = "wind speed inverted from model %s (%s)" % (GMF_VH_NAME, "VH")
dataset_1000m.windspeed_cr.attrs['model'] = GMF_VH_NAME
dataset_1000m.windspeed_cr.attrs['units'] = 'm/s'

illustration

In [ ]:
hv.Image(dataset_1000m.windspeed_co.compute(), label='wind speed co-pol').opts(cmap='jet',colorbar=True,clim=(0,80),height=100, width=125) + \
hv.Image(dataset_1000m.windspeed_cr, label='wind speed cr-pol').opts(cmap='jet',colorbar=True,clim=(0,80),height=100, width=125) + \
hv.Image(dataset_1000m.windspeed_dual.compute(), label='wind speed dual-pol').opts(cmap='jet',colorbar=True,clim=(0,80),height=100, width=125)

### save as a level-2 netcdf

delete useless variables

In [ ]:
# prepare dataset for netcdf export
black_list = ['model_U10', 'model_V10', 'digital_number', 'gamma0_raw', 'negz',
              'azimuth_time', 'slant_range_time', 'velocity', 'range_ground_spacing',
              'gamma0', 'time', 'sigma0', 'nesz', 'sigma0_raw', 'sigma0_ocean', 'altitude', 'elevation',
              'nd_co', 'nd_cr']

variables = list(set(dataset_1000m) - set(black_list))
dataset_1000m = dataset_1000m[variables]

remove complex

In [ ]:
dataset_1000m['ancillary_wind_spd'] = np.abs(dataset_1000m['ancillary_wind'])
dataset_1000m=dataset_1000m.assign(ancillary_wind_dir=(['line','sample'],np.angle(dataset_1000m['ancillary_wind']))) 
#dataset_1000m['ancillary_wind_dir'] = xr.ufuncs.angle(dataset_1000m['ancillary_wind'])
dataset_1000m['ancillary_wind_dir'].attrs['comment'] = 'angle in radians, anticlockwise, 0=xtrack'
del dataset_1000m['ancillary_wind']

In [ ]:
ds_1000 = dataset_1000m.compute()
ds_1000.attrs['footprint'] = str(xsar_obj_1000m.dataset.attrs['footprint'])
# encode gcps as json string
import json
class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)

json_gcps = json.dumps(json.loads(json.dumps(ds_1000.line.spatial_ref.gcps,cls=JSONEncoder)))
ds_1000['line']['spatial_ref'].attrs['gcps'] = json_gcps
ds_1000['sample']['spatial_ref'].attrs['gcps'] = json_gcps

In [ ]:
ds_1000

In [ ]:
#ds_1000.to_netcdf("my_L2_product")